# 데이터소스 API의 구조
## 읽기 API 구조
`DataFrameReader.format(...).option("key", "value").schema(...).load()`
* 읽을 경로를 지정해야 함
* `DataFrameReader`
  * `SparkSession`의 `read` 속성으로 접근
* `format` (포맷)
  * 선택적으로 사용 가능
  * 기본값은 parquet
* `schema` (스키마)
  * 데이터 소스에 스키마 제공하거나, 스키마 추론 기능 사용하려는 경우 선택적으로 사용
* `option` (옵션)
  * 데이터 읽는 방법에 대한 파라미터를 키-값 쌍으로 설정
  * 선택적으로 사용 가능
* 읽기 모드
  * 형식에 맞지 않는 데이터를 만났을 때 데이터 읽는 방식을 결정할 수 있는 옵션 제공
  * 설정값 가진 맵 객체 전달하는 등 다양한 방법 존재
  <table>
    <tr>
        <th>읽기 모드</th>
        <th>설명</th>
    </tr>
    <tr>
        <td>permissive</td>
        <td>오류 레코드의 모든 필드를 null로 설정하고, 모든 오류 레코드를 <code>_corrupt_record</code>라는 문자열 컬럼에 기록. 기본값.</td>
    </tr>
    <tr>
        <td>dropMalformed</td>
        <td>형식에 맞지 않는 레코드가 포함된 로우를 제거.</td>
    </tr>
    <tr>
        <td>failFast</td>
        <td>형식에 맞지 않는 레코드를 만나면 즉시 종료.</td>
    </tr>
  </table>
  
## 쓰기  API 구조
`DataFrameWriter.format(...).option(...).partitionBy(...).bucketBy(...).sortBy(...).save()`
* 저장될 경로를 입력해야 함
* `DataFrameWrtier`
  * `DataFrame`별로 접근
* `format`
  * 선택적으로 사용 가능
  * 기본값은 parquet
* `option`
  * 데이터 쓰기 방법 설정
* `partitionBy`, `bucketBy`,`sortBy`
  * 파일 기반의 데이터 소스에서만 동작
  * 파일 배치 형태를 제어
* 저장 모드
  * 지정된 위치에서 동일한 파일 발견했을 때 동작 방식 지정하는 옵션
  <table>
    <tr>
        <th>저장 모드</th>
        <th>설명</th>
    </tr>
    <tr>
        <td>append</td>
        <td>해당 경로에 이미 존재하는 파일 목록에 결과 파일 추가.</td>
    </tr>
    <tr>
        <td>overwrite</td>
        <td>이미 존재하는 모든 데이터를 완전히 덮어씀.</td>
    </tr>
    <tr>
        <td>errorIfExists</td>
        <td>해당 경로에 데이터나 파일이 존재하는 경우 오류 발생시키며 쓰기 작업 실패. 기본값.</td>
    </tr>
    <tr>
        <td>ignore</td>
        <td>해당 경로에 데이터나 파일이 존재하는 경우 아무런 처리하지 않음</td>
    </tr>
  </table>
  
# CSV
* 각 줄이 단일 레코드되며, 레코드의 각 필드를 콤마로 구분하는 일반적인 텍스트 파일 포맷
* 지연 연산 특성이 있어 DataFrame 정의 시점이 아닌 잡 실행 시점에만 오류 발생
* 쓰기 API 사용 시 여러 파일 들어 있는 디렉터리 생성
  * 데이터 쓰는 시점의 파티션 수 반영
  * 사전에 데이터 분할할 경우 파일 수 달라짐

## 옵션
![csv options](../../assets/presentations/week09/csv-options1.png)
![csv options](../../assets/presentations/week09/csv-options2.png)

# JSON
* 자바스크립트 객체 표기법
* 줄로 구분된 JSON 기본적으로 사용
  * `multiLine` 옵션 `true` 설정 시 전체 파일을 하나의 JSON 객체로 읽을 수 있음
  * 새로운 레코드 추가 가능
* 안정적인 포맷으로 권장
* 구조화 되었음
* 최소한의 기본 데이터 타입 존재

## 옵션
![json options](../../assets/presentations/week09/json-options1.png)
![json options](../../assets/presentations/week09/json-options2.png)


# Parquet
* 다양한 스토리지 최적화 기술을 제공하는 오픈소스로 만들어진 컬럼 기반 데이터 저장 방식
* 분석 워크로드에 최적화
* 저장 공간 절약 가능
* 전체 파일 읽는 대신 개별 칼럼 읽을 수 있음
* 컬럼 기반 압축 기능 제공
* 스파크와 잘 호환되기 때문에 스파크의 기본 파일 포맷이기도 함
* 복합 데이터 타입 지원
* 스키마가 파일 자체에 내장되어 읽는 시점에 스키마를 알 수 있음
* 파일 경로만 명시하면 쓰기 가능
* 분할 규칙은 다른 포맷과 동일하게 적용

## 옵션
![parquet options](../../assets/presentations/week09/parquet-options.png)

# ORC
* 하둡 워크로드를 위해 설계된 자기 기술적이며 데이터 타입을 인식할 수 있는 컬럼 기반의 파일 포맷
* 대규모 스트리밍 읽기에 최적화
* 필요한 로우를 신속하게 찾을 수 있는 기능 통합
* Parquet 포맷과 유사하나, Parquet는 스파크에 최적화된 반면 ORC는 하이브에 최적화

# SQL 데이터베이스
* 스파크 클래스패스에 데이터베이스의 JDBC 드라이버 추가하고 적절한 JDBC 드라이버 jar 파일을 제공해야 함
* 쿼리 푸시다운
  * DataFrame 만들기 전 데이터베이스 자체에서 데이터를 필터링하도록 만들 수 있음
* 데이터베이스 병렬로 읽기
  * `numPartitions` 옵션 사용해 읽기 및 쓰기 동시 작업 수 제한할 수 있는 최대 파티션 수를 설정할 수 있음
  * 명시적 조건절을 SQL 데이터베이스에 위임할 수 있음
    * 특정 파티션에 특정 데이터의 물리적 위치 제어 가능
* 슬라이딩 윈도우 기반의 파티셔닝
  * 조건절 기반으로 분할

## 옵션
![database options](../../assets/presentations/week09/database-options1.png)
![database options](../../assets/presentations/week09/database-options2.png)

# 텍스트 파일
* 각 줄은 DataFrame의 레코드
* 기본 데이터 타입의 유연성을 활용할 수 있음

# 고급 I/O 개념
* 쓰기 작업 전 파티션 수를 조절함으로써 병렬 처리할 파일 수를 제어할 수 있음
* 버켓팅과 파티셔닝 조절함으로써 데이터 저장 구조 제어할 수 있음
* 분할 가능 파일 타입과 압축 방식
  * 분할 지원 시 필요한 부분만 읽을 수 있어 성능 향상에 도움이 됨
  * Parquet 파일 포맷과 GZIP 압축 방식 권장
* 병렬로 데이터 읽기
  * 여러 익스큐터가 같은 파일을 동시에 읽을 수 없지만, 여러 파일을 동시에 읽을 수 있음
    * 개별 파일은 DataFrame의 파티션이 됨
* 병렬로 데이터 쓰기
  * 파일이나 데이터 수는 데이터 쓰는 시점에 DataFrame 가진 파티션 수에 따라 달라질 수 있음
    * 데이터 파티션당 하나의 파일 작성
    * 옵션에 지정된 파일명은 다수의 파일을 가진 디렉터리
* 파티셔닝
  * 어떤 데이터를 어디에 저장할 것인지 제어할 수 있는 기능
  * 파티셔닝된 디렉터리 또는 테이블에 파일을 쓸 때 디렉터리별로 컬럼 데이터를 인코딩해 저장
    * 필요한 컬럼의 데이터만 읽을 수 있음
  * 모든 파일 기반의 데이터소스에서 지원
* 버켓팅
  * 파일에 저장된 데이터를 제어할 수 있는 또 다른 파일 조직화 기법
  * 동일 버킷 ID를 가진 데이터가 하나의 물리적 파티션에 모두 모여 있어 데이터 셔플을 피할 수 있음
    * 조인이나 집계 시 발생하는 고비용 셔플을 피할 수 있음
* 복한 데이터 유형 쓰기
  * 모든 파일 포맷에 적합하진 않음
* 파일 크기 관리
  * 데이터 읽을 때 중요
    * 작은 파일을 많이 생성하면 메타 데이터 관리에 부하 발생
    * 몇 개 로우 필요하더라도 전체 데이터 블록 읽어야 하기 때문에 너무 큰 파일도 비효율적
  * 스파크 2.2 버전에는 자동으로 파일 크기 제어할 수 있는 새로운 방법 도입되었음
    * `maxRecordsPerFile` 옵션에 파일당 레코드 수 지정 가능